### SciPy versus NumPy

In [ ]:
# Import numpy symbols to scipy namespace
from numpy import *
from numpy.random import rand, randn
from numpy.fft import fft, ifft
from numpy.lib.scimath import *

In [ ]:
import numpy as np
a = np.identity(3)

### Statistics

#### Random Variables and Distributions

In [ ]:
%matplotlib inline
from scipy.stats import beta
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10,6)

q = beta(5, 5) # Beta(a, b), with a = b = 5
obs = q.rvs(2000) # 2000 observations
grid = np.linspace(0.01, 0.99, 100)

fig, ax = plt.subplots()
ax.hist(obs, bins=40, density=True)
ax.plot(grid, q.pdf(grid), 'k-', linewidth=2)
plt.show()

In [ ]:
q.cdf(0.4) # Cumulative distribution function

In [ ]:
q.ppf(0.8) # Quantile (inverse cdf) function

In [ ]:
q.mean()

#### Alternative Syntax

In [ ]:
obs = beta.rvs(5, 5, size=2000)
grid = np.linspace(0.01, 0.99, 100)

fig, ax = plt.subplots()
ax.hist(obs, bins=40, density=True)
ax.plot(grid, beta.pdf(grid, 5, 5), 'k-', linewidth=2)
plt.show()

In [ ]:
from scipy.stats import linregress

x = np.random.randn(200)
y = 2 * x + 0.1 * np.random.randn(200)
gradient, intercept, r_value, p_value, std_err = linregress(x, y)
gradient, intercept

### Roots and Fixed Points

In [ ]:
f = lambda x: np.sin(4 * (x - 1/4)) + x + x**20 - 1
x = np.linspace(0, 1, 100)

fig, ax = plt.subplots()
ax.plot(x, f(x), label='$f(x)$')
ax.axhline(ls='--', c='k')
ax.set_xlabel('$x$', fontsize=12)
ax.set_ylabel('$f(x)$', fontsize=12)
ax.legend(fontsize=12)
plt.show()

#### Bisection

In [ ]:
def bisect(f, a, b, tol=10e-5):
    """
    Implements the bisection root finding algorithm, assuming that f is a
    real-valued function on [a, b] satisfying f(a) < 0 < f(b).
    """

    lower, upper = a, b
    while upper - lower > tol:
        middle = 0.5 * (upper + lower)
        if f(middle) > 0: # root is between lower and middle
            lower, upper = lower, middle
        else: # root is between middle and upper
            lower, upper = middle, upper
    
    return 0.5 * (upper + lower)

In [ ]:
bisect(f, 0, 1)

In [ ]:
from scipy.optimize import bisect

bisect(f, 0, 1)

#### The Newton-Raphson Method

In [ ]:
from scipy.optimize import newton

newton(f, 0.2) # Start the search at initial condition x = 0.2

In [ ]:
newton(f, 0.7) # Start the search at x = 0.7 instead

#### Hybrid Methods

In [ ]:
from scipy.optimize import brentq

brentq(f, 0, 1)

In [ ]:
%timeit brentq(f, 0, 1)

In [ ]:
%timeit bisect(f, 0, 1)

#### Fixed Points

In [ ]:
from scipy.optimize import fixed_point

fixed_point(lambda x: x**2, 10.0) # 10.0 is an initial guess

### Optimization

In [ ]:
from scipy.optimize import fminbound

fminbound(lambda x: x**2, -1, 2) # Search in [-1, 2]

### Integration